In [1]:
from sympy import *
import numpy as np

In [2]:
def gradient_descent(f, sym, x0, alpha = 0.1, eps = 1e-4):
    df = np.array([100])
    i = 1
    it = 1
    while eps < np.linalg.norm(df):
        df = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(), f.diff(sym[1]).subs(zip(sym, x0)).evalf()]).astype(np.float64)
        y = x0 - alpha*df
        if f.subs(zip(sym,y)).evalf()<f.subs(zip(sym, x0)).evalf():
            x0 = y
        else:
            alpha = alpha/2
        i = i + 4
        it = it + 1
    return f.subs(zip(sym,x0)).evalf(), x0, i, it

def gradient_descent(f, sym, x0, alpha = 0.1, eps = 1e-4):
    df = np.array([100,133])
    i = 1
    it = 1
    x0 = ImmutableDenseNDimArray(x0)
    while eps < df[0]**2 + df[1]**2:
        df = tensor.derive_by_array(f,sym).subs(zip(sym,x0))
        y = x0 - df.applyfunc(lambda x: x*alpha)
        if f.subs(zip(sym,y)).evalf() < f.subs(zip(sym, x0)).evalf():
            x0 = y
        else:
            alpha = alpha/2
        i = i + 4
        it = it + 1
    return f.subs(zip(sym,x0)).evalf(), x0, i, it

In [3]:
def iterative_search(f, sym, a, b, eps=1e-6, delta = 1000):
    eps = eps
    x0 = a
    x1 = a+delta
    it = 0
    while eps<abs(delta):
        x1 = a+delta
        while f.subs({sym : x1}).evalf() < f.subs({sym : x0}).evalf() and x1>a and x1<b:
            x0 = x1
            x1 = x1+delta
            it = it + 1
        delta = -delta/4
    return f.subs({sym : x0}).evalf(), x0, it

In [4]:
def newton_search(f, sym, eps=1e-6):
    x0 = iterative_search(f,sym,0,oo,10)[1]
    it = 0 
    d = diff(f, sym).subs({sym : x0}).evalf()
    while eps < abs(d):
        x0 = x0 - d/diff(f, sym, 2).subs({sym : x0}).evalf()
        it = it + 2
        if it>200:
            return iterative_search(f,sym,0,oo)
        d = diff(f, sym).subs({sym : x0}).evalf()
    return f.subs({sym : x0}).evalf(), x0, it

In [5]:
def golden_split_search(f, sym, a, b, eps=1e-6):
    tau = (np.sqrt(5)-1)/2
    x2 = a + tau*(b-a)
    x1 = a+b-x2
    f1 = f.subs({sym: x1})
    f2 = f.subs({sym: x2})
    t = 0
    while eps < (b-a)/2: 
        if f.subs({sym: x1})<=f.subs({sym: x2}):
            b = x2
            x2 = x1
            x1 = a+b-x1
        else:
            a = x1
            x1 = x2
            x2 = a+b-x2
        t = t+1
    return f.subs({sym: (a+b)/2}), (a+b)/2, t

In [6]:
def full_descent(f, sym, x0, eps=1e-4, minimizer = newton_search):
    theta = symbols('theta')
    i = 0
    it = 0
    df = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(), f.diff(sym[1]).subs(zip(sym, x0)).evalf()]).astype(np.float64)
    while eps < np.linalg.norm(df):
        df = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(), f.diff(sym[1]).subs(zip(sym, x0)).evalf()]).astype(np.float64)
        f_ = f.subs(zip(sym, x0-theta*df))
        theta_min = minimizer(f_, theta)
        x0 = x0 - theta_min[1]*df
        i = i + 2 + theta_min[2]
        it = it + 1
    return f.subs(zip(sym,x0)).evalf(), x0, i, it

In [7]:
def ortogonal_descent(f, sym, x0, eps=1e-4, restart = 0, minimizer = lambda x,y: newton_search(x,y), beta = 'v2'):
    theta = symbols('theta')
    i = 0
    it = 0
    df = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(16), f.diff(sym[1]).subs(zip(sym, x0)).evalf(16)]).astype(np.float64)
    df_1 = 0     
    while eps < np.linalg.norm(df):
        y = df-df_1
        theta_min = minimizer(f.subs(zip(sym, x0-theta*df)), theta)
        x0 = x0 - theta_min[1]*df
        i = i + 2 + theta_min[2]
        s = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(16), f.diff(sym[1]).subs(zip(sym, x0)).evalf(16)]).astype(np.float64)
    
        if beta == 'v3':
            b = (np.linalg.norm(s)/np.linalg.norm(df))**2
        elif beta == 'v1':
            b = float(re(np.dot(y,s)/np.dot(y, df)))
        elif beta == 'v2':
            b = float(np.dot(y,s)/(np.linalg.norm(df)**2))
        else:
            raise Exception("Unknown beta type")
        
        df_1 = df
        df = np.array(-1*s + df*b)
        it = it+1
        if restart and it%restart == 0:
            df = np.array([f.diff(sym[0]).subs(zip(sym, x0)).evalf(16), f.diff(sym[1]).subs(zip(sym, x0)).evalf(16)]).astype(np.float64)
    return f.subs(zip(sym, x0)).evalf(16), x0, i, it

In [8]:
x1 = symbols('x1')
x2 = symbols('x2')
theta = symbols('theta')
def f(a):
    return x1**2+a*x2**2

# Задание 2

In [9]:
a = 1
eps = 1e-3
gradient_descent(f(a), [x1,x2], [1,-5], eps=eps), full_descent(f(a), [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((1.20423772806809e-7, array([ 6.80564734e-05, -3.40282367e-04]), 173, 44),
 (0, array([0, 0], dtype=object), 6, 2),
 (0, array([0, 0], dtype=object), 4, 1))

In [10]:
a = 1
eps = 1e-5
gradient_descent(f(a), [x1,x2], [1,-5], eps=eps), full_descent(f(a), [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((1.02445216110626e-11, array([ 6.27710174e-07, -3.13855087e-06]), 257, 65),
 (0, array([0, 0], dtype=object), 6, 2),
 (0, array([0, 0], dtype=object), 4, 1))

In [11]:
a = 250
eps = 1e-3
gradient_descent(f(a), [x1,x2], [1,-5], eps=eps), full_descent(f(a), [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((2.44849917188919e-7,
  array([ 4.94823117e-004, -2.22698617e-303]),
  4877,
  1220),
 (2.49837157123098e-8,
  array([0.000158062378978527, 5.05799612696839e-10], dtype=object),
  14,
  4),
 (9.947397318971396e-8,
  array([0.000315394946343812, 2.01047032472555e-9], dtype=object),
  18,
  5))

In [12]:
a = 250
eps = 1e-5
gradient_descent(f(a), [x1,x2], [1,-5], eps=eps), \
full_descent(f(a), [x1,x2], [1,-5], eps=eps, minimizer=lambda x,y: golden_split_search(x,y, 0, 10)),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps, minimizer=lambda x,y: newton_search(x,y,1e-8))

((2.46458570399349e-11,
  array([ 4.96445939e-006, -4.94065646e-324]),
  7813,
  1954),
 (1.85480468850578e-12, array([3.81713942e-07, 8.26825049e-08]), 420, 12),
 (3.478776037713882e-14,
  array([1.86514771066129e-7, 1.48497590613481e-12], dtype=object),
  34,
  9))

In [13]:
a = 1000
eps = 1e-3
gradient_descent(f(a), [x1,x2], [1,-5], 2, eps=eps), \
full_descent(f(a), [x1,x2], [1,-5], eps=eps), ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((2.48888469898506e-7, array([4.98887232e-04, 4.09995923e-81]), 15601, 3901),
 (1.59029673086385e-9,
  array([3.98785246818414e-5, 7.97570489470198e-12], dtype=object),
  14,
  4),
 (6.353819188516023e-9,
  array([7.97108473640912e-5, 3.18524712356917e-11], dtype=object),
  18,
  5))

In [14]:
a = 1000
eps = 1e-5
gradient_descent(f(a), [x1,x2], [1,-5], eps=eps),\
full_descent(f(a), [x1,x2], [1,-5], eps=eps,minimizer = lambda x,y: iterative_search(x,y,0,oo,1e-8)),\
ortogonal_descent(f(a), [x1,x2], [1,-5], eps=eps)

((2.49049400608615e-11,
  array([4.99048495e-006, 4.94065646e-324]),
  31257,
  7815),
 (1.62339895324017e-11, array([4.02146471e-06, 7.86200649e-09]), 18536, 6002),
 (1.012390119767756e-12,
  array([1.27134958014078e-8, -3.18155384488816e-8], dtype=object),
  102,
  46))

# Задание 3

In [15]:
f = 64*x1**2 + 126*x1*x2 + 64*x2**2 + -10*x1+30*x2 + 13

In [16]:
eps = 1e-3
gradient_descent(f, [x1,x2], [1,-5], eps=eps), full_descent(f, [x1,x2], [1,-5], eps=eps),\
ortogonal_descent(f, [x1,x2], [1,-5], eps=eps)

((-187.393700546578, array([  9.96028292, -10.03902307]), 3169, 793),
 (-187.393700784118,
  array([9.96058941390778, -10.0393295534509], dtype=object),
  46,
  12),
 (-187.3937007173045,
  array([9.96044284962962, -10.0391827405086], dtype=object),
  162,
  41))

# Задание 5

In [17]:
f = 100*(x1**2-x2)**2 + (x1-1)**2
eps = 1e-3
ortogonal_descent(f, [x1,x2], [-1,1], eps=eps, restart=2, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-8))

(1.449594811183616e-6, array([1.00120316, 1.00241224]), 1524, 541)

In [18]:
eps = 1e-5
ortogonal_descent(f, [x1,x2], [-1,1], restart=5, eps=eps)

(9.410831486762589e-7,
 array([0.999031133874847, 0.998058326307610], dtype=object),
 2216,
 486)

# Задание 6

In [19]:
eps = 1e-3
ortogonal_descent(f, [x1,x2], [-1, 1], eps=eps, restart=1, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-8))

(8.252310691103049e-7, array([1.00090775, 1.00181982]), 2208, 542)

In [20]:
ortogonal_descent(f, [x1,x2],  [-1, 1], eps=eps,restart=2,  minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-8))

(1.449594811183616e-6, array([1.00120316, 1.00241224]), 1524, 541)

In [21]:
eps = 1e-3
ortogonal_descent(f, [x1,x2],  [-1, 1], eps=eps,restart=3, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-8))

(8.303642004037187e-6, array([0.99711913, 0.99424007]), 1308, 401)

In [22]:
eps = 1e-3
ortogonal_descent(f, [x1,x2], [-1, 1], eps=eps, restart=4, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-8))

(5.132800170756114e-6, array([1.00226504, 1.00453032]), 967, 338)

In [23]:
eps = 1e-3
ortogonal_descent(f, [x1,x2],  [-1, 1], eps=eps, restart=5, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-8))

(9.904257769490771e-6, array([0.99685615, 0.99370788]), 6832, 2519)

# Задание 7

In [24]:
f = 64*x1**2 + 126*x1*x2 + 64*x2**2 + -10*x1+30*x2 + 13
ortogonal_descent(f, [x1,x2], [2, 4], eps=eps, restart=2, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-4))

(-187.3937006050728, array([  9.96032724, -10.03906926]), 2906, 795)

In [25]:
ortogonal_descent(f, [x1,x2], [2, 4], eps=eps, restart=2, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-6))

(-187.3937006050728, array([  9.96032724, -10.03906926]), 2906, 795)

In [26]:
ortogonal_descent(f, [x1,x2], [2, 4], eps=eps, restart=2, minimizer=lambda x,y: iterative_search(x,y,0,oo,1e-8))

(-187.3937006050710, array([  9.96032723, -10.03906926]), 2927, 795)

In [27]:
ortogonal_descent(f, [x1,x2],[2, 4], eps=eps, restart=2, minimizer=lambda x,y: golden_split_search(x,y,0,1000,1e-4))

(-187.3937005757780, array([  9.96030425, -10.03904525]), 4935, 141)

In [28]:
ortogonal_descent(f, [x1,x2], [2, 4], eps=eps, restart=2, minimizer=lambda x,y: golden_split_search(x,y,0,1000,1e-6))

(-187.3937005807584, array([  9.96030909, -10.03904815]), 1764, 42)

In [29]:
ortogonal_descent(f, [x1,x2], [2, 4], eps=eps, restart=2, minimizer=lambda x,y: golden_split_search(x,y,0,1000,1e-8))

(-187.3937007870372, array([  9.96061639, -10.03935679]), 3740, 85)

# Задание 8

In [30]:
f = (x1**2 + x2 - 11)**2 + (x1 + x2**2 -7)**2
gradient_descent(f, [x1,x2], [0, 0], eps=eps)

(5.17370747955689e-9, array([2.99999232, 2.00001853]), 109, 28)

In [31]:
full_descent(f, [x1,x2], [0, 0], eps=eps)

(9.12152992951906e-11,
 array([-2.80511662740639, 3.13131175402231], dtype=object),
 162,
 16)

In [32]:
ortogonal_descent(f, [x1,x2], [0, 0], eps=eps)

(1.166759836827425e-9,
 array([-3.77931124327955, -3.28318128268068], dtype=object),
 184,
 29)

In [33]:
gradient_descent(f, [x1,x2], [-5, 0], eps=eps)

(5.14034276799558e-9, array([ 3.58442662, -1.84810768]), 137, 35)

In [34]:
full_descent(f, [x1,x2], [-5, 0], eps=eps)

(4.36173365773719e-13,
 array([-2.80511797130789, 3.13131250931209], dtype=object),
 48,
 7)

In [35]:
ortogonal_descent(f, [x1,x2], [-5, 0], eps=eps)

(3.674845771682115e-9,
 array([-3.77930200621876, -3.28318415471229], dtype=object),
 138,
 23)